In [ ]:
!git clone https://github.com/italodellagarza/GCNMoneyLaundering

Cloning into 'GCNMoneyLaundering'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 64 (delta 28), reused 55 (delta 19), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [ ]:
%cd GCNMoneyLaundering/test_amlsim_nenn/

/content/GCNMoneyLaundering/test_amlsim_nenn


In [ ]:
!nvidia-smi

Wed Mar 30 11:48:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch; print(torch.__version__)

1.10.0+cu111


In [ ]:
pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu111.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 5.2 MB/s 
     |████████████████████████████████| 3.5 MB 24.0 MB/s 
     |████████████████████████████████| 2.5 MB 27.7 MB/s 
     |████████████████████████████████| 750 kB 12.6 MB/s 
     |████████████████████████████████| 407 kB 5.5 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=be329fcbc832c27dbabe67752090b04ddce926e54c9a837e11a57ff64f1ae333
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [ ]:
import os
from torch_geometric.data import Data
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dataset = []

for i in range(200):
    dataset.append(torch.load(f'/content/gdrive/MyDrive/Materiais Dissertação/Datasets/10Kvertices-1Medges-undersampling_new/{i}.pt'))

In [ ]:
train_data = dataset[0:int(0.8*200)]
test_data = dataset[int(0.8*200):]

In [ ]:
dataset[1].edge_to_edge_adj_matr.count_nonzero()

tensor(17)

In [ ]:
dataset[1].y

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0])

In [ ]:
dataset[1].edge_attr

tensor([[0.1066, 0.0518, 0.0587, 0.0381, 0.6754, 0.8840, 0.7796, 0.7899, 0.8066,
         0.3582, 0.2062, 0.8724, 0.9817, 0.1740, 0.0826, 0.9699, 0.9923],
        [0.1445, 0.0988, 0.1453, 0.1546, 0.0867, 0.1781, 0.1331, 0.1223, 0.1219,
         0.1381, 0.1473, 0.0940, 0.0862, 0.1095, 0.1382, 0.1328, 0.0916],
        [0.1447, 0.0990, 0.1456, 0.1549, 0.0858, 0.1770, 0.1320, 0.1212, 0.1208,
         0.1378, 0.1473, 0.0927, 0.0847, 0.1095, 0.1384, 0.1314, 0.0901],
        [0.0755, 0.0914, 0.0658, 0.0795, 0.0870, 0.1106, 0.0814, 0.0498, 0.0588,
         0.1073, 0.0610, 0.0511, 0.1064, 0.1058, 0.0918, 0.1076, 0.0965],
        [0.0752, 0.0910, 0.0655, 0.0791, 0.0875, 0.1113, 0.0819, 0.0504, 0.0595,
         0.1074, 0.0609, 0.0519, 0.1073, 0.1056, 0.0914, 0.1084, 0.0974]],
       dtype=torch.float64)

In [ ]:
len(train_data)

160

In [ ]:
len(test_data)

40

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from model_nenn_simplificado import Nenn

In [ ]:
dataset[0].x.shape[0]

3

In [ ]:
dataset[0].edge_attr.shape[0]

5

In [ ]:
model = Nenn(3,5,3,5,2)

In [ ]:
model = model.cuda()

In [ ]:
loss = torch.nn.CrossEntropyLoss().cuda()
# TODO Undersampling Smote
# TODO Oversampling
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
dataset[0]

Data(x=[3, 48], edge_index=[2, 24], edge_attr=[5, 24], y=[24], node_to_node_adj_matr=[48, 48], node_to_edge_adj_matr=[48, 24], edge_to_node_adj_matr=[24, 48], edge_to_edge_adj_matr=[24, 24])

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
def train():
    model.train()
    # Enumerar sobre os dados.
    for ts, data in enumerate(train_data):
        if not len(data.x) == 0:
          for epoch in range(1000):
              # Resetar Gradientes
              optimizer.zero_grad()
              # Passando as informações do batch e de conexão dos grafos
              logits = model(
                  data.x.T.type(torch.FloatTensor).cuda(),
                  data.edge_attr.T.type(torch.FloatTensor).cuda(),
                  data.edge_to_edge_adj_matr.type(torch.FloatTensor).cuda(),
                  data.edge_to_node_adj_matr.type(torch.FloatTensor).cuda(),
                  data.node_to_edge_adj_matr.type(torch.FloatTensor).cuda(),
                  data.node_to_node_adj_matr.type(torch.FloatTensor).cuda()
              )
              #label_pred = pred.max(1)[1]
              # Calculando a perda e os gradientes.
              l = loss(logits.cuda(), data.y.cuda())
              l.backward()
              # Atualizar usando os gradientes.
              optimizer.step()
              if (epoch+1) % 100 == 0:
                  print('ts',ts+1,'epoch =', epoch + 1, 'loss =', l.item())
              torch.cuda.empty_cache() 

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
train()

ts 1 epoch = 100 loss = 0.6681815981864929
ts 1 epoch = 200 loss = 0.653384268283844
ts 1 epoch = 300 loss = 0.6396763920783997
ts 1 epoch = 400 loss = 0.6270509958267212
ts 1 epoch = 500 loss = 0.6154742240905762
ts 1 epoch = 600 loss = 0.6048843860626221
ts 1 epoch = 700 loss = 0.5952203869819641
ts 1 epoch = 800 loss = 0.5864254832267761
ts 1 epoch = 900 loss = 0.5784291625022888
ts 1 epoch = 1000 loss = 0.5711591839790344
ts 2 epoch = 100 loss = 0.6252073645591736
ts 2 epoch = 200 loss = 0.6239079833030701
ts 2 epoch = 300 loss = 0.6226557493209839
ts 2 epoch = 400 loss = 0.6214559078216553
ts 2 epoch = 500 loss = 0.6203124523162842
ts 2 epoch = 600 loss = 0.619227409362793
ts 2 epoch = 700 loss = 0.6181758046150208
ts 2 epoch = 800 loss = 0.6168614029884338
ts 2 epoch = 900 loss = 0.6157605648040771
ts 2 epoch = 1000 loss = 0.6147642731666565
ts 3 epoch = 100 loss = 0.7929322123527527
ts 3 epoch = 200 loss = 0.7802165150642395
ts 3 epoch = 300 loss = 0.7689554691314697
ts 3 epoch 

In [ ]:
label_pred_list = []
y_true_list = []

def test():
    model.eval()
    with torch.no_grad():
        global label_pred_list
        global y_true_list
        
        for data in test_data:
            data.to('cuda')
            print(data.node_to_edge_adj_matr.shape)
            logits = model(
                data.x.T.type(torch.FloatTensor).cuda(),
                data.edge_attr.T.type(torch.FloatTensor).cuda(),
                data.edge_to_edge_adj_matr.type(torch.FloatTensor).cuda(),
                data.edge_to_node_adj_matr.type(torch.FloatTensor).cuda(),
                data.node_to_edge_adj_matr.type(torch.FloatTensor).cuda(),
                data.node_to_node_adj_matr.type(torch.FloatTensor).cuda()
            )
            label_pred = logits.max(1)[1].tolist()
            label_pred_list += label_pred
            y_true_list += data.y.tolist()
    model.train()

In [ ]:
torch.diag_embed(torch.rand([481, 443])).shape

torch.Size([481, 443, 443])

In [ ]:
test()

torch.Size([12, 6])
torch.Size([12, 6])
torch.Size([16, 8])
torch.Size([17, 9])
torch.Size([20, 10])
torch.Size([16, 8])
torch.Size([10, 5])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([18, 9])
torch.Size([12, 6])
torch.Size([14, 7])
torch.Size([14, 7])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([20, 10])
torch.Size([21, 11])
torch.Size([18, 9])
torch.Size([8, 4])
torch.Size([13, 7])
torch.Size([12, 6])
torch.Size([8, 4])
torch.Size([12, 6])
torch.Size([16, 8])
torch.Size([7, 4])
torch.Size([10, 5])
torch.Size([16, 8])
torch.Size([8, 4])
torch.Size([22, 11])
torch.Size([10, 5])
torch.Size([12, 6])
torch.Size([4, 2])
torch.Size([6, 3])
torch.Size([10, 5])
torch.Size([8, 4])
torch.Size([13, 7])
torch.Size([22, 11])
torch.Size([10, 5])
torch.Size([11, 6])
torch.Size([6, 3])


In [ ]:
label_pred_list

[0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,


In [ ]:
y_true_list

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
mean_precision = []
mean_recall = []
mean_f1 = []

prec = precision_score(y_true_list, label_pred_list, average='binary', labels=[0])
rec = recall_score(y_true_list, label_pred_list, average='binary', labels=[0])
f1 = f1_score(y_true_list, label_pred_list, average='binary', labels=[0])

In [ ]:
print(classification_report(y_true_list,label_pred_list))

              precision    recall  f1-score   support

           0       0.97      0.74      0.84       220
           1       0.31      0.84      0.46        31

    accuracy                           0.75       251
   macro avg       0.64      0.79      0.65       251
weighted avg       0.89      0.75      0.79       251



In [ ]:
prec

0.3132530120481928

In [ ]:
rec

0.8387096774193549

In [ ]:
f1

0.456140350877193